In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
from torchvision.models import resnet18
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import copy
import pathlib

base_path = pathlib.Path("/home/mpuscian/Desktop/repozytoria/MINI_projects/anvil/models/")
cifar_model_path = base_path.joinpath("cifar_model.pth")


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = resnet18(weights=None)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
model.fc = nn.Linear(512, 10)

adaround = copy.deepcopy(model)

adaround.load_state_dict(torch.load(cifar_model_path))
adaround.eval().to(device)

print(f"Using device: {device}")

# 2. Transforms
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

# 3. Datasets
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_size = int(0.9 * len(full_train_dataset))  # 45,000
val_size = len(full_train_dataset) - train_size  # 5,000
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# 4. DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=6)
val_dataset.dataset.transform = transform_test
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=6)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=6)

Using device: cuda


In [2]:
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np

val_dataset = val_loader.dataset  # your full val dataset

num_samples = len(val_dataset)
subset_size = num_samples // 5

# Randomly sample 1/5 indices
all_indices = np.arange(num_samples)
np.random.shuffle(all_indices)
subset_indices = all_indices[:subset_size]

# Create a sampler using those indices
sampler = SubsetRandomSampler(subset_indices)

# Create DataLoader with this sampler
subset_loader = DataLoader(
    val_dataset,
    batch_size=val_loader.batch_size,
    sampler=sampler,
    num_workers=val_loader.num_workers,
    pin_memory=True
)

In [3]:
from aimet_torch.adaround.adaround_weight import Adaround, AdaroundParameters
from aimet_torch.quantsim import QuantizationSimModel
from aimet_common.defs import QuantScheme

def calibration_data_loader():
    for i, (x, _) in enumerate(val_loader):
        yield x.to(device)
        if i >= 9:  # Limit to 10 batches, as specified in AdaroundParameters
            break


params = AdaroundParameters(
    data_loader=subset_loader,
    num_batches=min(10, len(subset_loader)),
    default_num_iterations=3000,
    default_reg_param=0.07,
    default_beta_range=(40, 2)
)

dummy_input = torch.randn(1, 3, 32, 32).to(device)

adaround = Adaround.apply_adaround(
    model=adaround,
    dummy_input=dummy_input,
    params=params,
    path=base_path.joinpath('aimet/adaround_encodings'),
    filename_prefix='resnet18',
    default_param_bw=6,
    default_quant_scheme=QuantScheme.post_training_tf_enhanced
)

/usr/lib64/python3.13/abc.py:106: FutureWarning: `NLLLoss2d` has been deprecated. Please use `NLLLoss` instead as a drop-in replacement and see https://pytorch.org/docs/main/nn.html#torch.nn.NLLLoss for more details.
  cls = super().__new__(mcls, name, bases, namespace, **kwargs)


2025-06-21 12:25:37,336 - ConnectedGraph - WARNING - Unable to isolate model outputs.
2025-06-21 12:25:37,354 - Quant - INFO - Unsupported op type Squeeze
2025-06-21 12:25:37,354 - Quant - INFO - Unsupported op type Mean
2025-06-21 12:25:37,355 - Quant - INFO - Unsupported op type Unsqueeze
2025-06-21 12:25:37,355 - Quant - INFO - Unsupported op type Compress
2025-06-21 12:25:37,355 - Quant - INFO - Unsupported op type Identity
2025-06-21 12:25:37,355 - Quant - INFO - Unsupported op type Shape
2025-06-21 12:25:37,355 - Quant - INFO - Unsupported op type If
2025-06-21 12:25:37,357 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:None
2025-06-21 12:25:38,232 - ConnectedGraph - WARNING - Unable to isolate model outputs.
2025-06-21 12:25:38,336 - Utils - INFO - Caching 8 batches from data loader at path location: /tmp/tmp70akwl8y


2025-06-21 12:25:38,342 - Quant - INFO - Started Optimizing weight rounding of module: conv1


  0%|                                                                                                                                                                                               | 0/60 [00:00<?, ?it/s]/home/mpuscian/Envs/Optimization/lib64/python3.13/site-packages/aimet_torch/_base/adaround/adaround_optimizer.py:257: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  return float(recons_err_hard), float(recons_err_soft)


2025-06-21 12:25:40,631 - Quant - INFO - Started Optimizing weight rounding of module: layer1.0.conv1


...

2025-06-21 12:26:23,353 - Quant - INFO - Started Optimizing weight rounding of module: layer4.1.conv1


2025-06-21 12:26:27,769 - Quant - INFO - Started Optimizing weight rounding of module: layer4.1.conv2


2025-06-21 12:26:32,202 - Quant - INFO - Started Optimizing weight rounding of module: fc


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:56<00:00,  1.07it/s]

2025-06-21 12:26:34,613 - Quant - INFO - Completed Adarounding Model


In [4]:
print(f"Model first layer device: {next(adaround.parameters()).device}")
print(f"Dummy input device: {dummy_input.device}")

Model first layer device: cuda:0
Dummy input device: cuda:0


In [4]:
def forward_pass(model, _ =  None):
    for batch in calibration_data_loader():
        model(batch)

sim = QuantizationSimModel(
    model=adaround,
    dummy_input=dummy_input,
    quant_scheme=QuantScheme.post_training_tf_enhanced,
    default_output_bw=4,
    default_param_bw=4,
)
sim.compute_encodings(forward_pass)

2025-06-21 12:27:10,116 - ConnectedGraph - WARNING - Unable to isolate model outputs.
2025-06-21 12:27:10,134 - Quant - INFO - Unsupported op type Squeeze
2025-06-21 12:27:10,134 - Quant - INFO - Unsupported op type Mean
2025-06-21 12:27:10,134 - Quant - INFO - Unsupported op type Unsqueeze
2025-06-21 12:27:10,135 - Quant - INFO - Unsupported op type Compress
2025-06-21 12:27:10,135 - Quant - INFO - Unsupported op type Identity
2025-06-21 12:27:10,135 - Quant - INFO - Unsupported op type Shape
2025-06-21 12:27:10,135 - Quant - INFO - Unsupported op type If
2025-06-21 12:27:10,136 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:None


# Evaluation

In [5]:
def evaluate_model(model, dataloader, device):
    model = model.to(device)
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.append(predicted.cpu())
            all_labels.append(labels.cpu())
    
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    
    return all_labels, all_preds

In [6]:
from sklearn import metrics

In [8]:
unquantized_model = copy.deepcopy(model)

unquantized_model.load_state_dict(torch.load(cifar_model_path))
unquantized_model = unquantized_model.eval().to(device)

In [10]:
y_true, y_pred = evaluate_model(sim.model, test_loader, device)
print(f"Adaround (testset) 4 bits AIMET: {metrics.accuracy_score(y_true, y_pred)}")

Adaround (testset) 4 bits AIMET: 0.7668


In [7]:
y_true, y_pred = evaluate_model(sim.model, test_loader, device)
print(f"Adaround (testset) 6 bits AIMET: {metrics.accuracy_score(y_true, y_pred)}")

Adaround (testset) 6 bits AIMET: 0.7659


In [11]:
torch.unique(sim.model.conv1.weight)

tensor([-0.2497, -0.2374, -0.2355,  ...,  0.2353,  0.2356,  0.2504],
       device='cuda:0', grad_fn=<Unique2Backward0>)